In [2]:
%pylab inline
from scipy import special


Populating the interactive namespace from numpy and matplotlib


In [19]:
special.gamma(3.2)

2.4239654799353687

In [102]:
def xi_transformation(r):
    return (r-1)/(r+1)

def Anl(n, l):
    knl = 0.5*n*(n+4*l+3) + (l+1)*(2*l+1)
    A_nl = - 2**(8*l+6)/(4*np.pi*knl) * (special.factorial(n)*(n+2*l+3/2.)*(special.gamma(2*l+3/2.))**2)/(special.gamma(n+4*l+3))
    return A_nl

def phi_nl(n, l, r):
    xi = xi_transformation(r)
    return - special.eval_gegenbauer(n, 2*l + 3/2., xi)* (4*np.pi)**(0.5)*r**l / ((1+r)**(2*l+1))  

In [104]:
def snl(n, l, m, m_p, r_p, theta_p, phi_p):
    A_nl = Anl(n,l)
    if m==0:
        S_nl = A_nl * np.sum(m_p*phi_nl(n, l, r_p)*special.sph_harm(m, l, 0, theta_p)*np.cos(m*phi_p))
    if m!=0:
        S_nl = 2*A_nl * np.sum(m_p*phi_nl(n, l, r_p)*special.sph_harm(m, l, 0, theta_p)*np.cos(m*phi_p))
    return S_nl.real

In [24]:
# data:
spherical_halo = genfromtxt('../data/spherical_halo.txt')

In [25]:
x = spherical_halo[:,0]
y = spherical_halo[:,1]
z = spherical_halo[:,2]
m = spherical_halo[:,0]

In [27]:
r = (x**2 + y**2 + z**2)**0.5
theta = np.arccos(z/r)
phi = np.arctan2(y, x)


In [109]:
snl(15, 0, 0, m, r, theta, phi)

-22.69757350897029

In [72]:
max(theta)

3.1377070881166214

In [74]:
special.sph_harm(0, 1, 0., theta)

array([ 0.32574794+0.j,  0.33413773+0.j,  0.34700852+0.j, ...,
        0.03430162+0.j,  0.02743251+0.j,  0.06379421+0.j])